In [ ]:
import pandas as pd
import numpy as np
from sklearn import *
import xgboost as xgb
import re

In [2]:
train=pd.read_csv('/home/manish/Desktop/Data2017/February/lending/train_indessa.csv')
test=pd.read_csv('/home/manish/Desktop/Data2017/February/lending/test_indessa.csv')

In [3]:
params={"objective":"binary:logistic",     
    "learning_rate":0.1,
    "subsample":0.8,
    "colsample_bytree": 0.8,
        'eval_metric':'auc',
    "max_depth":6,
    'silent':1,
    'nthread':3,
#     'num_class':2
        
       }

In [4]:
params_lgb = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'application': 'binary',
    'metric':'auc',
    'num_leaves': 128,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_threads':3,
    'max_bin':500,
    'verbose': -1
}

In [5]:
def last_week(string):
    string =re.findall('\d+',string )
    if len(string)==0:
        return -1
    return int(string[0])

In [6]:
data=train.append(test)
data.reset_index(drop=1,inplace=1)

In [7]:
data.last_week_pay=data.last_week_pay.apply(lambda x:last_week(x) )


In [8]:
data.loc[data.term=='36 months','term']=36*4
data.loc[data.term=='60 months','term']=60*4

In [9]:
data.term.head()

0    144
1    144
2    144
3    144
4    144
Name: term, dtype: object

In [10]:
data['week_left']=data.last_week_pay/data.term

data['same_bank']=0
data.loc[data.loan_amnt!=data.funded_amnt,'same_bank']=1


data['same_investor']=0
data.loc[data.loan_amnt!=data.funded_amnt_inv,'same_investor']=1

data['total_loan']=data.funded_amnt*(data.int_rate/100)*3+data.funded_amnt
data.loc[data.term==240,'total_loan']=data.funded_amnt*(data.int_rate/100)*5+data.funded_amnt

data['loan_left']=(data.total_loan*(data.last_week_pay/data.term)+data.total_rec_int)/data.total_loan


data['inc_loan_ratio']=data.funded_amnt/data.annual_inc


data['monthly_loan']=(data.dti/100)*(data.annual_inc/12)
data['loan_total']=(data.dti/100)*(data.annual_inc/12)*data.term

data['loan_to_income']=(data.monthly_loan*12)/data.annual_inc


data['interest_month']=data.total_rec_int/data.last_week_pay
data.interest_month.replace(np.inf,-1,inplace=1)


data['tot_rec_int_ratio']=data.total_rec_int/(data.funded_amnt*(data.int_rate/100)*3)


data.loc[data.term==240,'tot_rec_int_ratio']=data.total_rec_int/(data.funded_amnt*(data.int_rate/100)*5)


data['left_interest']=1-data.tot_rec_int_ratio


data.total_rev_hi_lim=data.revol_bal/(data.revol_util/100)
data.total_rev_hi_lim.replace(np.inf,-1,inplace=1)


df_temp=data.groupby('batch_enrolled').agg({'loan_amnt':
                                            'mean'}).rename(columns={'loan_amnt':'batch_mean'}).reset_index()
data=pd.merge(data,df_temp,on='batch_enrolled',how='left')

df_temp=data.groupby('sub_grade').agg({'loan_amnt':
                                            'mean'}).rename(columns={'loan_amnt':'sg_mean'}).reset_index()
data=pd.merge(data,df_temp,on='sub_grade',how='left')

In [11]:
df_temp=pd.get_dummies(data['grade'],prefix='grade')
for col in df_temp.columns:
    data[col]=df_temp[col]
    
df_temp=pd.get_dummies(data['sub_grade'],prefix='sub_grade')
for col in df_temp.columns:
    data[col]=df_temp[col]

In [12]:
features=list(set(data.columns)-set(['emp_title','desc','title','verification_status_joint','loan_status',
                                     'member_id','total_loan','loan_left','loan_total'
                                     ]))

In [15]:
for col in features:
    lr=preprocessing.LabelEncoder()
    if data[col].dtype=='O':
        data[col].replace(np.nan,'AA',inplace=1)
        lr.fit(data[col])
        data[col]=lr.transform(data[col])
    data[col].replace(np.nan,-1,inplace=1)
    data[col].replace(np.inf,-1,inplace=1)

In [16]:
df_temp=data.groupby('batch_enrolled').agg({'last_week_pay':
                                             'value_counts'}).rename(columns={'last_week_pay':
                                                                              'week_counts'}).reset_index()

df_temp_1=df_temp.groupby('batch_enrolled').agg({'week_counts':'sum'}).rename(columns={'week_counts':
                                                                              'week_sum'}).reset_index()

df_temp=pd.merge(df_temp,df_temp_1,on='batch_enrolled',how='left')

df_temp.week_counts=df_temp.week_counts/df_temp.week_sum

data=pd.merge(data,df_temp[['batch_enrolled','last_week_pay','week_counts']],on=['batch_enrolled','last_week_pay'],
              how='left')


In [17]:
df_temp=data.last_week_pay.value_counts().reset_index().rename(columns={'last_week_pay':'week_ratio'})
df_temp.week_ratio=df_temp.week_ratio/df_temp.week_ratio.sum()

In [18]:
data['week_counts_remain']=1-data.week_counts

In [ ]:
train=data[~(data.loan_status.isnull())]
test=data[data.loan_status.isnull()]
test.drop('loan_status',axis=1,inplace=1)

In [20]:
df_temp=train.groupby('last_week_pay').agg({'loan_status':
                                            'mean'}).rename(columns={'loan_status':'fraction_default'}).reset_index()
train=pd.merge(train,df_temp,on='last_week_pay',how='left')
test=pd.merge(test,df_temp,on='last_week_pay',how='left')

In [21]:
features=list(set(train.columns)-set(['emp_title','desc','title','verification_status_joint','loan_status',

                                      'member_id','total_loan','batch_enrolled','loan_total','week_counts_remain',
                                      'tot_rec_int_ratio','interest_month','monthly_loan','inc_loan_ratio',
                                      'initial_list_status','loan_left','zip_code','pymnt_plan','tot_coll_amt','application_type'
                                      ,'funded_amnt_inv','purpose','delinq_2yrs','inq_last_6mths'
                                     ]))

In [22]:
total_index=train[train.loan_status==0].index

sampled_train1=train.iloc[np.random.choice(total_index,len(train[train.loan_status==1]),replace=False)]
sampled_train1=sampled_train1.append(train[train.loan_status==1])
sampled_train1=sampled_train1.iloc[np.random.permutation(len(sampled_train1))]

total_index=list(set(total_index)-set(list(sampled_train1.index)))

In [50]:
# lr=ensemble.RandomForestClassifier(n_estimators=50,random_state=2016)
# lr.fit(x_train,y_train)
# cv=lr.predict_proba(x_cv)[:,1]

# lst=lr.predict_proba(test[features])[:,1]
# print(metrics.log_loss(y_cv,lr.predict_proba(x_cv)[:,1]))
# cv_score=lr.feature_importances_

KeyboardInterrupt: 

In [ ]:
x_train,x_cv,y_train,y_cv=cross_validation.train_test_split(train[features],train.loan_status,random_state=0)

dtrain=xgb.DMatrix(x_train.values,y_train)
dvalid=xgb.DMatrix(x_cv.values,y_cv)
dtest=xgb.DMatrix(test[features].values)
watchlist = [ (dtrain, 'train'),(dvalid, 'cv')]

gbm=xgb.train(params,dtrain,6000,evals=watchlist,early_stopping_rounds=50,verbose_eval=1);
cv=(gbm.predict(dvalid))
lst=(gbm.predict(dtest))

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_cv, y_cv, reference=lgb_train)
gbm = lgb.train(params_lgb,lgb_train,num_boost_round=6000,valid_sets=lgb_eval,early_stopping_rounds=50)

In [ ]:
df_imp=pd.DataFrame(list(gbm.get_score().items()),columns=['feature','importance'])
df_imp.feature=df_imp.feature.apply(lambda x: features[last_week(x)])
df_imp.sort(columns='importance',ascending=0,inplace=1)

In [61]:
# cv_lgb=gbm.predict(x_cv)
lst_lgb=gbm.predict(test[features])

In [54]:
df=pd.DataFrame()
df['xgb']=cv
df['lgb']=cv_lgb

In [64]:
df['loan_status']=y_cv.values

In [71]:
metrics.roc_auc_score(y_cv,(df.xgb+df.lgb)/2)

0.98121848186411154

In [66]:
df_test=pd.DataFrame()
df_test['xgb']=lst
df_test['lgb']=lst_lgb

In [24]:
def model(train,test,features,params,isknn,islr,):    
    cv=[]
    truth=[]
    cv_scores=[]
    lst=[]
#     x_train,y_train,x_cv,y_cv=get_data(train,features)
    skf=cross_validation.StratifiedKFold(train.loan_status,n_folds=5,random_state=0)
    for idx1,idx2 in skf:
        
        x_train,x_cv=train[features].iloc[idx1],train[features].iloc[idx2]
        y_train,y_cv=train.loan_status.iloc[idx1],train.loan_status.iloc[idx2]
        truth.extend(y_cv)
       
        if isknn:
            lr=ensemble.RandomForestClassifier(n_estimators=70,random_state=2016)
            lr.fit(x_train,y_train)
            cv.extend(lr.predict_proba(x_cv)[:,1])
            
            lst.append(lr.predict_proba(test[features])[:,1])
            print(metrics.roc_auc_score(y_cv,lr.predict_proba(x_cv)[:,1]))
            cv_scores.append(lr.feature_importances_)



        elif islr:

            lr=neighbors.KNeighborsClassifier(n_neighbors=10,n_jobs=3)
            lr.fit(x_train,y_train)
            cv.extend(lr.predict_proba(x_cv)[:,1])
            
            lst.append(lr.predict_proba(test[features])[:,1])
            print(metrics.roc_auc_score(y_cv,lr.predict_proba(x_cv)[:,1]))
            cv_scores.append(np.absolute(lr.coef_))
    
            
        else:

            dtrain=xgb.DMatrix(x_train.values,y_train)
            dvalid=xgb.DMatrix(x_cv.values,y_cv)
            dtest=xgb.DMatrix(test[features].values)
            watchlist = [ (dtrain, 'train'),(dvalid, 'cv')]

            gbm=xgb.train(params,dtrain,6000,evals=watchlist,early_stopping_rounds=30,verbose_eval=.1);
            cv.extend(gbm.predict(dvalid))
            cv_scores.append(metrics.log_loss(y_cv,gbm.predict(dvalid)))
            lst.append(gbm.predict(dtest))
    print ('Overall',metrics.roc_auc_score(truth,np.array(cv)))
    return cv,lst,np.average(cv_scores,axis=0),truth


In [ ]:
output=model(train,test,features,params,0,0)

In [ ]:
cv_preds['knn']=output[0]
test_preds['knn']=np.average(output[1],axis=0)

In [ ]:
cv_preds=pd.read_csv('cv.csv')
test_preds=pd.read_csv('test.csv')

In [43]:
# cv_preds=pd.DataFrame()
# test_preds=pd.DataFrame()

# cv_preds['xgb_1']=output[0]
# cv_preds['loan_status']=output[-1]
# test_preds['xgb_1']=output[1]

In [47]:
cv_preds['rf_1']=output[0]
test_preds['rf_1']=np.average(output[1],axis=0)

In [73]:
ls=[1 if x>.5 else 0 for x in cv_preds.xgb_1]

In [70]:
# output=model(train,test,features,params,0,1)
cv_preds['rf_1']=output[0]

test_preds_2['rf_1']=np.average(output[1],axis=0)

In [23]:
results=pd.DataFrame()
results['member_id']=test.member_id
results['loan_status']=np.average(output[1],axis=0)
results.to_csv('output.csv',index=0)